https://twice22.github.io/rl-part4/

In [1]:
import gym
env=gym.make("Blackjack-v1")

/home/jay_boo/.local/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/jay_boo/.local/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
print(env.observation_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))


This tuples rpresent the states :

&rarr;The first element is the current player's sum $ {0,1,....,31 } $

&rarr; The second is the dealer's face up card ${1,...,10}$

&rarr; The plays has an usuale ace $0,1$. O:NO , 1:YES



In [3]:
env.reset(return_info=True)

((14, 1, False), {})

En terme d'action possible : Soit on retire une carte, soit l'on ne fait rien :

 - 1 : HIT. On tire une carte
 - 2 : Stick. ON ne tire pas de carte 

In [4]:
env.action_space

Discrete(2)

In [5]:
env.step(1)

((19, 1, False), 0.0, False, {})

Let launch 4 parties with random agent

In [6]:

for _ in range(4):
    state=env.reset()
    while True :
        print(state)
        a=env.action_space.sample()
        state,reward,term,info=env.step(a)
        if term :
            print("win" if reward >0 else "lose"+"\n")
            break

(19, 9, False)
(21, 9, False)
lose

(7, 8, False)
lose

(11, 6, False)
win
(20, 10, False)
lose



In [7]:
def generate_episode_deterministic(env):
    state=env.reset()
    episode=[]
    while True:
        action=int(state[0]<=18)#Our deterministic policy
        nextState,reward,term,info=env.step(action)
        episode.append((state,action,reward))
        state=nextState
        if term :
            break
    return episode

Monte-Carlo  Value estimation first visit:


In [8]:
def MCV(env,generate_episode,gamma,episodes):
    V,returns={},{}

    for episode in range(episodes):
        trajectory = generate_episode(env)
        G,T=0,len(trajectory)
        visited_state=set()

        for i in range(T-1,-1,-1):
            state,action,reward=trajectory[i]
            G+=gamma**i * reward
            if state not in visited_state :
                reward, visits = returns.get(state,[0,0])
                returns[state]= [reward + G ,visits +1]
                visited_state.add(state)